# Installing and Importing and Initializing Libraries.

In [ ]:
!pip install pandarallel

In [1]:
import plotly.express as px
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [2]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
#from pandarallel import pandarallel

pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#pandarallel.initialize(progress_bar=True)

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


In [ ]:
df=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')

# Windowing the training data
### As the target is to detect an event, 
### So, extracting timesteps 30 min before and after the events. 

In [3]:
def window(df, win_size):
    ind=df.index[df['event'].isna()==False]
    c=0
    df['window']=np.nan
    for i in tqdm(ind):
        a=i-win_size
        b=i+win_size
        df['window'].loc[a:i]=int(c)
        c=c+1
        df['window'].loc[i:b]=int(c)
        c=c+1
    df['window'].dropna(inplace=True)
    return df[df['window'].isna()==False]

#df_series=df_series[df_series['window'].isna()==False]

# Removing the steps where change in enmo and anglez is insignificant (device is supposed not to be worn)

In [4]:
def inactive_periods(df):
    print("shape before application: ",df.shape)
    df['diff_anglez']=df['anglez'].diff()
    df=df[(df['enmo']!=0.0) | (df['diff_anglez']!=0.0)]
    print("shape after application: ",df.shape)
    df.drop('diff_anglez', inplace=True, axis=1)
    print("shape after completion: ",df.shape)
    print("removed ")
    return df


# Clustering the Enmo and Anglez

In [5]:
def clustering(df):
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler    
    X=df[['anglez','enmo']]
#Scalling the data
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(X)
#perform clustering
    model=KMeans(n_clusters=4,algorithm="elkan" )
    model.fit(X_scaled)
    return model.labels_


# Adding Rolling Standard Deviations

- As per observation enmo and anglez varies frequently and more than normal for few timesteps before and after the event.
- In order to consider the variations into the modelling, using the method.

In [7]:
 def rollingstd(series_df):
# Creating columns with nans
    series_df['sd_enmo_1']=np.nan    # 1 min rolling std: enmo
    series_df['sd_anglez_1']=np.nan  # 1 min rolling std: anglez
    series_df['m_enmo_2']=np.nan     # 2 min rolling mean: enmo
    series_df['m_anglez_2']=np.nan   # 2 min rolling std: anglez 
    print('anglez rolling std 12')
    series_df['sd_anglez_1'] = (series_df.groupby('series_id')['anglez']
                      .rolling(12)
                      .std()
                      .reset_index(level=0, drop=True))
    print('anglez rolling std 2')
    series_df['sd_anglez_1'][series_df['sd_anglez_1'].isna()==True] = (series_df.groupby('series_id')['anglez']
                      .rolling(2)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling std 12')
    series_df['sd_enmo_1'] = (series_df.groupby('series_id')['enmo']
                      .rolling(12)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling mean 24')
    series_df['m_enmo_2'] = (series_df.groupby('series_id')['enmo']
                      .rolling(24)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('anglez rolling mean 24')
    series_df['m_anglez_2'] = (series_df.groupby('series_id')['anglez']
                      .rolling(24)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('enmo rolling std 2')
    print('Nans in sd_emno_1: ',series_df['sd_enmo_1'].isnull().sum())
    series_df['sd_enmo_1'][series_df['sd_enmo_1'].isna()==True] = (series_df.groupby('series_id')['enmo']
                      .rolling(2)
                      .std()
                      .reset_index(level=0, drop=True))
    print('enmo rolling mean 2')
    series_df['m_enmo_2'][series_df['m_enmo_2'].isna()==True] = (series_df.groupby('series_id')['enmo']
                      .rolling(2)
                      .mean()
                      .reset_index(level=0, drop=True))
    print('anglez rolling mean 2')
    series_df['m_anglez_2'][series_df['m_anglez_2'].isna()==True] = (series_df.groupby('series_id')['anglez']
                      .rolling(2)
                      .mean()
                      .reset_index(level=0, drop=True))
#Series wise rolling std and mean
# filling rest of nans
    print('Nans in sd_emno_1: ',series_df['sd_enmo_1'].isnull().sum())
    series_df['sd_enmo_1'].fillna(0.0, inplace=True)
    series_df['sd_anglez_1'].fillna(0.0, inplace=True)
    series_df['m_enmo_2'].fillna(0.0, inplace=True)
    series_df['m_anglez_2'].fillna(0.0, inplace=True)
    print('Nans after removal: ',series_df['sd_enmo_1'].isnull().sum())

    return(series_df)


# Scaling the data

In [8]:
def scale(X):
    from sklearn import preprocessing
    scaler=preprocessing.StandardScaler().fit(X)
    return (scaler.transform(X))


# Creating Training Data

In [9]:
# Importing the datasets
print('Importing Training Datasets')
df_series=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet", engine='auto')
df_events=pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
print('Dataset Imported...')
print('______________________________________')

# Merging the datasets
print('Merging the training datasets...')
events=df_events[['series_id', 'step','event']]
series_df=pd.merge(df_series,events,on=["step","series_id"],how='left')
df_series=[]
df_event=[]
series_df['sleep']=np.nan
series_df.loc[series_df["event"]=="onset", "sleep"] = 1
series_df.loc[series_df["event"]=="wakeup", "sleep"] = 0
series_df['sleep'].fillna(method='ffill', inplace=True)
series_df['sleep'].fillna(value=0, inplace=True)
print('Datasets Merged...')
print('______________________________________')

# Removing the periods of inactivity
print('Removing the periods of Inactivity...')
series_df=inactive_periods(series_df)
print('______________________________________')

# Forming Windows
win_size=720  #600mins
print('Creating Windows each size: ',win_size)
series_df=window(series_df,win_size)
print('Windows formed...')
print('______________________________________')

# Adding the columns of Standard Deviation (1 min)
print('Adding columns to account for deviation in enmo and anglez 1 min rolling...')
series_df=rollingstd(series_df)
series_df['sd_anglez_1']=pd.to_numeric(series_df['sd_anglez_1'])
series_df['sd_enmo_1']=pd.to_numeric(series_df['sd_enmo_1'])
series_df['m_anglez_2']=pd.to_numeric(series_df['m_anglez_2'])
series_df['m_enmo_2']=pd.to_numeric(series_df['m_enmo_2'])
print('Std columns added...')
print('______________________________________')

# Clustering the Data
print('Clustering the data based on enmo and anglez...')
series_df['cluster']=(clustering(series_df)+1)/4
print('Added clusters...')





Importing Training Datasets
Dataset Imported...
______________________________________
Merging the training datasets...
Datasets Merged...
______________________________________
Removing the periods of Inactivity...
shape before application:  (127946340, 7)
shape after application:  (111766109, 8)
shape after completion:  (111766109, 7)
removed 
______________________________________
Creating Windows each size:  720


100%|██████████| 6928/6928 [00:04<00:00, 1434.39it/s]


Windows formed...
______________________________________
Adding columns to account for deviation in enmo and anglez 1 min rolling...
anglez rolling std 12
anglez rolling std 2
enmo rolling std 12
enmo rolling mean 24
anglez rolling mean 24
enmo rolling std 2
Nans in sd_emno_1:  2959
enmo rolling mean 2
anglez rolling mean 2
Nans in sd_emno_1:  269
Nans after removal:  0
Std columns added...
______________________________________
Clustering the data based on enmo and anglez...
Added clusters...


In [ ]:
series_df.dtypes

In [10]:
figure= px.imshow(series_df[['sd_anglez_1','sd_enmo_1','m_anglez_2','m_enmo_2','anglez','enmo','cluster','sleep']].corr(),text_auto=True, width=1200, height=1200)
figure.show()

# Creating Train Test Data from the Training Data

In [11]:
from sklearn.model_selection import train_test_split
X=series_df[['sd_anglez_1','sd_enmo_1','anglez','m_anglez_2','m_enmo_2','enmo','cluster']]
y_test=series_df[['sleep']]
X_test=scale(X)
#X_train, X_test, y_train, y_test =train_test_split(X_scaled,y,test_size=0.2, random_state=42)
del X


In [12]:
y_test[['sleep']].value_counts()

sleep
0.0      4803476
1.0      3766687
Name: count, dtype: int64

# Function to evaluate the training (scikit-Learn)

In [14]:
def evaluate(y_test,ypred):
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm = confusion_matrix(y_test, y_pred)
    figure= px.imshow(cm,text_auto=True, width=1200, height=1200)
    figure.show()


In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', 
              max_iter =100, 
              alpha=10, 
              hidden_layer_sizes=5, 
              random_state=5,
              activation='relu',
              batch_size=360, 
              learning_rate='adaptive', 
              verbose=1,
              early_stopping=1, 
              n_iter_no_change=10)

print ('Training the model')
clf.fit(X_train,y_train)
print(clf.score(X_train,y_train))
print ('Saving the model')
from joblib import dump, load
dump(clf, 'mlp_model.joblib')
y_pred=clf.predict(X_test)
evaluate(y_test,y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_test,y_test)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')
y_pred=rf.predict(X_test)
evaluate(y_test,y_pred)
X_test=[]

In [15]:
from joblib import dump, load
model=load('rf_model.joblib')
y_pred=model.predict(X_test)
evaluate(y_test,y_pred)
X_test=[]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   48.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  1.9min finished


Accuracy:  0.9599386849468324
Precision Score :  0.9654582323449457
Recall Score:  0.9580646120049299
F1 Score:  0.9538785801891164


In [35]:
result_df=series_df[['series_id', 'step','timestamp']]


NameError: name 'df_series' is not defined

In [20]:
result_df['sleep']=y_pred

In [21]:
result_df

,series_id,step,timestamp,sleep
4272,038441c925bb,4272,2018-08-14T21:26:00-0400,0.0
4273,038441c925bb,4273,2018-08-14T21:26:05-0400,0.0
4274,038441c925bb,4274,2018-08-14T21:26:10-0400,0.0
4275,038441c925bb,4275,2018-08-14T21:26:15-0400,0.0
4276,038441c925bb,4276,2018-08-14T21:26:20-0400,0.0
...,...,...,...,...
127928942,fe90110788d2,574982,2017-09-07T00:05:10-0400,1.0
127929176,fe90110788d2,575216,2017-09-07T00:24:40-0400,1.0
127929177,fe90110788d2,575217,2017-09-07T00:24:45-0400,1.0
127929178,fe90110788d2,575218,2017-09-07T00:24:50-0400,1.0


In [22]:
result_df['timestamp']=result_df[['timestamp']].progress_apply(lambda x: pd.to_datetime(x,utc=True))

100%|██████████| 1/1 [00:39<00:00, 39.80s/it]


In [33]:
import datetime as dt
result_df=result_df.groupby(['series_id',result_df.timestamp.dt.floor('30Min')])['sleep'].mean()

In [34]:
result_df.reset_index 

series_id     timestamp                
038441c925bb  2018-08-15 01:00:00+00:00    0.041667
              2018-08-15 01:30:00+00:00    0.000000
              2018-08-15 02:00:00+00:00    0.148459
              2018-08-15 02:30:00+00:00    1.000000
              2018-08-15 03:00:00+00:00    0.984026
                                             ...   
fe90110788d2  2017-09-06 03:00:00+00:00    0.804196
              2017-09-07 02:30:00+00:00    0.000000
              2017-09-07 03:00:00+00:00    0.000000
              2017-09-07 03:30:00+00:00    0.754808
              2017-09-07 04:00:00+00:00    0.909091
Name: sleep, Length: 33946, dtype: float64

In [ ]:
model_nn=tf.keras.Sequential([
  tf.keras.layers.Dense(360,input_shape=[X_train.shape[1]]),
  tf.keras.layers.Dense(360,activation=tf.nn.leaky_relu, use_bias=True),
  tf.keras.layers.Dense(180,activation=tf.nn.relu),
  tf.keras.layers.Dense(90,activation=tf.nn.relu, use_bias=True),
  tf.keras.layers.Dense(20,activation=tf.nn.leaky_relu),
  tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])
#model_nn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001) , 
#                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
#                metrics=['accuracy'])



In [ ]:
epochs = 100
batch_size = win_size

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.x", save_best_only=True, monitor="val_loss"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=epochs, min_lr=0.0001
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=epochs, verbose=1),
]
model_nn.compile(
    optimizer="adam",
    #loss="sparse_categorical_crossentropy",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['binary_accuracy'],
)
model_nn.summary()

In [ ]:
y_train.dtypes

In [ ]:
history = model_nn.fit(
    X_train,
    y_train,
    batch_size=360,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
model = tf.keras.models.load_model("best_model.x")


In [ ]:
metric = "binary_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

In [ ]:
model = tf.keras.models.load_model("best_model.x")

test_loss, test_acc = model.evaluate(X_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
test.isnull().sum()

In [9]:
from joblib import dump, load
model=load('rf_model.joblib')

In [11]:
test=pd.read_parquet(path="/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet", engine='auto')
#test_df=test
test_df=rollingstd(test)
test_df['cluster']=(clustering(test_df)+1)/4
test_df['sd_anglez_1']=pd.to_numeric(test_df['sd_anglez_1'])
test_df['sd_enmo_1']=pd.to_numeric(test_df['sd_anglez_1'])
X=test_df[['sd_anglez_1','sd_enmo_1','anglez','m_anglez_2','m_enmo_2','enmo','cluster']]
y=model.predict(scale(X))


100%|██████████| 3/3 [00:00<00:00, 49.59it/s]

Nans in sd_emno_1:  3
Nans after removal:  0



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [95]:
test['series_id'].unique()

array(['038441c925bb', '03d92c9f6f8a', '0402a003dae9'], dtype=object)

In [98]:
test[test['series_id']=='0402a003dae9']

,series_id,step,timestamp,anglez,enmo,sd_enmo_1,sd_anglez_1,m_enmo_2,m_anglez_2,cluster
300,0402a003dae9,0,2018-12-18T12:45:00-0500,-77.314903,0.0868,0.000000,0.000000,0.000000,0.000000,0.50
301,0402a003dae9,1,2018-12-18T12:45:05-0500,-52.183800,0.0434,17.770374,17.770374,0.000000,0.000000,0.50
302,0402a003dae9,2,2018-12-18T12:45:10-0500,-53.218601,0.0541,0.731715,0.731715,0.000000,0.000000,0.50
303,0402a003dae9,3,2018-12-18T12:45:15-0500,-51.541698,0.0612,1.185749,1.185749,0.000000,0.000000,0.50
304,0402a003dae9,4,2018-12-18T12:45:20-0500,-24.637300,0.0591,19.024282,19.024282,0.000000,0.000000,0.25
305,0402a003dae9,5,2018-12-18T12:45:25-0500,-23.642300,0.0483,0.703572,0.703572,0.000000,0.000000,0.25
306,0402a003dae9,6,2018-12-18T12:45:30-0500,-18.802099,0.0744,3.422539,3.422539,0.000000,0.000000,0.25
307,0402a003dae9,7,2018-12-18T12:45:35-0500,-16.594200,0.0657,1.561220,1.561220,0.000000,0.000000,0.25
308,0402a003dae9,8,2018-12-18T12:45:40-0500,-22.122499,0.0604,3.909098,3.909098,0.000000,0.000000,0.25
309,0402a003dae9,9,2018-12-18T12:45:45-0500,-21.585400,0.0433,0.379787,0.379787,0.000000,0.000000,0.25


In [12]:
result_df=test_df[['series_id','step','timestamp']]

In [100]:
ls

best_model.x/  mlp_model.joblib  rf_model.joblib


In [ ]:
result_df[['sleep']]

In [13]:
print(result_df.shape)
print(y.shape)
result_df[['sleep']]=np.nan
result_df[['sleep']]=result_df[['sleep']].apply(lambda x: y)

(450, 3)
(450,)


In [14]:
result_df[['timestamp']]=result_df[['timestamp']].apply(lambda x: pd.to_datetime(x,utc=True))

In [15]:
result_df['series_id'].unique()

array(['038441c925bb', '03d92c9f6f8a', '0402a003dae9'], dtype=object)

In [16]:
result_df['diff']=np.nan
for series_id in tqdm(result_df['series_id'].unique()):
    result_df['diff'][result_df['series_id']==series_id]=result_df['sleep'].diff()
    

    


100%|██████████| 3/3 [00:00<00:00, 363.15it/s]


In [17]:
result_df['diff'].value_counts()

diff
 0.0    439
 1.0      5
-1.0      5
Name: count, dtype: int64

In [19]:
result_df['event']=np.nan
result_df.loc[result_df["diff"]==1, "event"] = 'onset'
result_df.loc[result_df["diff"]==-1, "event"] = 'wakeup'

In [20]:
result_df['event'].value_counts()

event
onset     5
wakeup    5
Name: count, dtype: int64

In [24]:
result_df.loc[(result_df["step"]==0) & (result_df["sleep"]==1), "event"] = 'onset'
result_df.loc[(result_df["step"]==0) & (result_df["sleep"]==0), "event"] = 'wakeup'
result_df.loc[(result_df["step"]==result_df['step'].groupby(result_df['series_id']).max()) & (result_df["sleep"]==1), "event"] = 'onset'
result_df.loc[(result_df["step"]==result_df['step'].groupby(result_df['series_id']).max()) & (result_df["sleep"]==0), "event"] = 'wakeup'

In [77]:
for series_id in tqdm(result_df['series_id'].unique():
    maxstep=result_df['step'][result_df['series_id']==series_id].max()
    result_df.loc[(result_df["step"]==maxstep) & (result_df["sleep"]==1), "event"] = 'onset'
    result_df.loc[(result_df["step"]==maxstep) & (result_df["sleep"]==0), "event"] = 'wakeup'

100%|██████████| 450/450 [00:00<00:00, 523.27it/s]


In [78]:
result_df

,series_id,step,timestamp,sleep,diff,event
0,038441c925bb,0,2018-08-14 19:30:00+00:00,0.0,NaN,wakeup
1,038441c925bb,1,2018-08-14 19:30:05+00:00,0.0,0.0,NaN
2,038441c925bb,2,2018-08-14 19:30:10+00:00,0.0,0.0,NaN
3,038441c925bb,3,2018-08-14 19:30:15+00:00,0.0,0.0,NaN
4,038441c925bb,4,2018-08-14 19:30:20+00:00,0.0,0.0,NaN
5,038441c925bb,5,2018-08-14 19:30:25+00:00,0.0,0.0,NaN
6,038441c925bb,6,2018-08-14 19:30:30+00:00,0.0,0.0,NaN
7,038441c925bb,7,2018-08-14 19:30:35+00:00,0.0,0.0,NaN
8,038441c925bb,8,2018-08-14 19:30:40+00:00,0.0,0.0,NaN
9,038441c925bb,9,2018-08-14 19:30:45+00:00,0.0,0.0,NaN


In [79]:
summary=result_df[result_df['event'].isna()==False]
summary.reset_index()


,index,series_id,step,timestamp,sleep,diff,event
0,0,038441c925bb,0,2018-08-14 19:30:00+00:00,0.0,NaN,wakeup
1,39,038441c925bb,39,2018-08-14 19:33:15+00:00,1.0,1.0,onset
2,149,038441c925bb,149,2018-08-14 19:42:25+00:00,1.0,0.0,onset
3,150,03d92c9f6f8a,0,2018-05-31 16:00:00+00:00,0.0,-1.0,wakeup
4,182,03d92c9f6f8a,32,2018-05-31 16:02:40+00:00,1.0,1.0,onset
5,183,03d92c9f6f8a,33,2018-05-31 16:02:45+00:00,0.0,-1.0,wakeup
6,184,03d92c9f6f8a,34,2018-05-31 16:02:50+00:00,1.0,1.0,onset
7,185,03d92c9f6f8a,35,2018-05-31 16:02:55+00:00,0.0,-1.0,wakeup
8,189,03d92c9f6f8a,39,2018-05-31 16:03:15+00:00,1.0,1.0,onset
9,190,03d92c9f6f8a,40,2018-05-31 16:03:20+00:00,0.0,-1.0,wakeup


# deleting the events with timeperiods smaller than 30 mins.

In [93]:
import datetime as dt
summary['timediff']=summary.groupby(summary['series_id'])['timestamp'].diff().dt.total_seconds().div(60)
summary                                                

,series_id,step,timestamp,sleep,diff,event,timediff
0,038441c925bb,0,2018-08-14 19:30:00+00:00,0.0,NaN,wakeup,NaN
39,038441c925bb,39,2018-08-14 19:33:15+00:00,1.0,1.0,onset,3.250000
149,038441c925bb,149,2018-08-14 19:42:25+00:00,1.0,0.0,onset,9.166667
150,03d92c9f6f8a,0,2018-05-31 16:00:00+00:00,0.0,-1.0,wakeup,NaN
182,03d92c9f6f8a,32,2018-05-31 16:02:40+00:00,1.0,1.0,onset,2.666667
183,03d92c9f6f8a,33,2018-05-31 16:02:45+00:00,0.0,-1.0,wakeup,0.083333
184,03d92c9f6f8a,34,2018-05-31 16:02:50+00:00,1.0,1.0,onset,0.083333
185,03d92c9f6f8a,35,2018-05-31 16:02:55+00:00,0.0,-1.0,wakeup,0.083333
189,03d92c9f6f8a,39,2018-05-31 16:03:15+00:00,1.0,1.0,onset,0.333333
190,03d92c9f6f8a,40,2018-05-31 16:03:20+00:00,0.0,-1.0,wakeup,0.083333


# Calculating the Score

In [63]:
summary['score']=np.nan
for i in tqdm(np.arange(0,len(summary)-1)):
    if i<len(summary)-1:
        next_index=summary['index'].index[i+1]
        next_series=summary['series_id'].index[i+1]
        this_series=summary['series_id'].index[i]
        this_index=summary['index'].index[i]
        summary['score'].index[i]=result_df['sleep'].loc[this_index:(next_index-1)].mean()
    if summary['event'].index[i]=='wakeup':
        summary['score'].index[i]=1-summary['score'].index[i]  #reversing the score for the wakeup events.
    else:
        print('completed')
            

39
150
182
183
184
185
189
190
195
300


In [45]:
summary['score']=np.nan
summary

,series_id,step,timestamp,sleep,diff,event,ind,score
0,038441c925bb,0,2018-08-14 19:30:00+00:00,0.0,NaN,wakeup,0,NaN
39,038441c925bb,39,2018-08-14 19:33:15+00:00,1.0,1.0,onset,39,NaN
150,03d92c9f6f8a,0,2018-05-31 16:00:00+00:00,0.0,-1.0,wakeup,150,NaN
182,03d92c9f6f8a,32,2018-05-31 16:02:40+00:00,1.0,1.0,onset,182,NaN
183,03d92c9f6f8a,33,2018-05-31 16:02:45+00:00,0.0,-1.0,wakeup,183,NaN
184,03d92c9f6f8a,34,2018-05-31 16:02:50+00:00,1.0,1.0,onset,184,NaN
185,03d92c9f6f8a,35,2018-05-31 16:02:55+00:00,0.0,-1.0,wakeup,185,NaN
189,03d92c9f6f8a,39,2018-05-31 16:03:15+00:00,1.0,1.0,onset,189,NaN
190,03d92c9f6f8a,40,2018-05-31 16:03:20+00:00,0.0,-1.0,wakeup,190,NaN
195,03d92c9f6f8a,45,2018-05-31 16:03:45+00:00,1.0,1.0,onset,195,NaN


In [46]:
len(summary)

11

In [47]:
summary.reset_index()

,index,series_id,step,timestamp,sleep,diff,event,ind,score
0,0,038441c925bb,0,2018-08-14 19:30:00+00:00,0.0,NaN,wakeup,0,NaN
1,39,038441c925bb,39,2018-08-14 19:33:15+00:00,1.0,1.0,onset,39,NaN
2,150,03d92c9f6f8a,0,2018-05-31 16:00:00+00:00,0.0,-1.0,wakeup,150,NaN
3,182,03d92c9f6f8a,32,2018-05-31 16:02:40+00:00,1.0,1.0,onset,182,NaN
4,183,03d92c9f6f8a,33,2018-05-31 16:02:45+00:00,0.0,-1.0,wakeup,183,NaN
5,184,03d92c9f6f8a,34,2018-05-31 16:02:50+00:00,1.0,1.0,onset,184,NaN
6,185,03d92c9f6f8a,35,2018-05-31 16:02:55+00:00,0.0,-1.0,wakeup,185,NaN
7,189,03d92c9f6f8a,39,2018-05-31 16:03:15+00:00,1.0,1.0,onset,189,NaN
8,190,03d92c9f6f8a,40,2018-05-31 16:03:20+00:00,0.0,-1.0,wakeup,190,NaN
9,195,03d92c9f6f8a,45,2018-05-31 16:03:45+00:00,1.0,1.0,onset,195,NaN


In [ ]:
result_df['score']=np.nan
result_df['score']=result_df['sleep'].loc[:iloc[results_df['event'].isna()==False.mean()